In [46]:
import sys
sys.path.append("..")
sys.path.append("../experiments")

from experiments.constants import * 
from experiments.utils import *

import pandas as pd
import json

In [47]:
SRC, TGT = ENGLISH, MALTESE

In [48]:
batches_filepath = f"/netscratch/falcao/data/{SRC}-{TGT}/eval-set/batches.{SRC}-{TGT}.json"
with open(filepath, mode="r") as f:
    j = json.load(f)

In [52]:
df_filepath = f"/netscratch/falcao/data/{SRC}-{TGT}/eval-set/df_eval.csv"
df_eval = pd.read_csv(df_filepath)

In [49]:
dfs = []
for batch_json in j:
    _df = pd.DataFrame(batch_json["items"])
    _df["_batchNo"] = batch_json["task"]["batchNo"]
    dfs.append(_df)

df = pd.concat(dfs)
df = df[["_batchNo", *df.columns[:-1]]]

In [50]:
print("total items:", len(df))

total items: 1600


In [44]:
df.sample(5, random_state=1)

,_batchNo,_block,_item,itemID,itemType,sourceID,sourceText,targetID,targetText
75,1,7,75,322,TGT,src.en,"Suits are standard business attire, and cowork...",kurt.mt,"suits huma attire tan-negozju standard, u l-ko..."
84,13,128,84,363,TGT,src.en,Men are naturally timid.,etranslate.mt,l-irġiel huma naturalment timida.
8,5,40,8,283,BAD,src.en,"With 17,000 islands to choose from, Indonesian...",nllb.mt,which extends beyond a traveller's previous il...
82,13,128,82,390,TGT,src.en,Several large television screens were installe...,etranslate.mt,Diversi skrins tat-televiżjoni kbar ġew instal...
47,15,144,47,216,TGT,src.en,"Eventually, after thousands or even millions o...",nllb.mt,"Fl- aħħar, wara eluf jew anke miljuni taʼ snin..."


In [45]:
identical_hyps = df[df.targetID.apply(lambda id: "+" in id)]
print("identical hypotheses:", len(identical_hyps))
identical_hyps

identical hypotheses: 0


,_batchNo,_block,_item,itemID,itemType,sourceID,sourceText,targetID,targetText


In [37]:
print("sourceID values:", set(df.sourceID))
assert len(set(df.sourceID)) == 1 # ? not sure

sourceID values: {'src.es'}


In [38]:
print("targetID values:", df.targetID.unique().tolist())

targetID values: ['cmbacktrans.eu', 'euskadi.eu', 'nllb.eu', 'ref.eu', 'cmbacktrans.eu+euskadi.eu']


In [39]:
print("itemType values:", set(df.itemType))
assert set(df.itemType) <= set(["BAD", "CHK", "REF", "TGT"])

print("value counts:")
print(df.itemType.value_counts().to_string())
print()

print("per batch:")
_df = df[df._batchNo == 1]
print(_df.itemType.value_counts().to_string())

itemType values: {'TGT', 'REF', 'BAD'}
value counts:
TGT    1200
BAD     150
REF     150

per batch:
TGT    80
BAD    10
REF    10
